# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import fowlkes_mallows_score as f

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y[Y.columns[0]].unique()), random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = f(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [5]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , fowlkes_mallows_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , fowlkes_mallows_score

(3, 0.9256603549576449)
(9, 0.9057845361495204)
(18, 0.8271652164870829)
(22, 0.7920642833993302)
(12, 0.7885531069166981)
(1, 0.784654889170588)
(19, 0.7823548697763435)
(6, 0.782214900190495)
(11, 0.7821084575227055)
(32, 0.7815590999641648)
(27, 0.7812882629691875)
(23, 0.7809181954505313)
(13, 0.7807158460475327)
(14, 0.7800959020266208)
(15, 0.7800908073922788)
(28, 0.7800721800415619)
(30, 0.7789923148798213)
(2, 0.7766461097867118)
(7, 0.7745267566092071)
(16, 0.7734300099933041)
(17, 0.7724030808333162)
(29, 0.7697406171868212)
(0, 0.7400710759831362)
(8, 0.7278019933617608)
(5, 0.7191461814460094)
(4, 0.7104534220088461)
(33, 0.7080015962025912)
(20, 0.7079990018256839)
(21, 0.6966804114081449)
(31, 0.6923536279882361)
(34, 0.6923536279882361)
(35, 0.6923536279882361)
(26, 0.6415142247278875)
(10, 0.5219931745439337)
(38, 0.3781640276269932)
(36, 0.3463662213872411)
(37, 0.3317622826610476)
(41, 0.325778751806409)
(39, 0.31305108665617

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [8]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6667    0.1777    0.2806       619
      Backdoor     0.6049    0.0864    0.1512       567
           DoS     0.3159    0.2719    0.2923      4075
      Exploits     0.6339    0.8081    0.7105     11236
       Fuzzers     0.8953    0.8639    0.8793      6127
       Generic     0.9973    0.9858    0.9915     53886
Reconnaissance     0.9274    0.7430    0.8250      3385
     Shellcode     0.6888    0.6160    0.6503       388
         Worms     0.8889    0.2105    0.3404        38

      accuracy                         0.8905     80321
     macro avg     0.7355    0.5293    0.5690     80321
  weighted avg     0.8943    0.8905    0.8878     80321



In [9]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [10]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.89719408002241 max accuracy
34 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [12]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('service', 0.28261089539463774)
('sbytes', 0.254637707622255)
('smeansz', 0.14653073582851903)
('Sjit', 0.09767825139808364)
('dmeansz', 0.08960935313847819)
('Spkts', 0.08400778878378806)
('Dload', 0.04176356634821849)
('state', 0.0020003144413110775)
('dttl', 0.0011613870447087506)


In [14]:
clf.score(x_train, y_train) 

0.9217549975488861

In [15]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.89719408002241

In [16]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.6250    0.1515    0.2439       495
      Backdoor     0.8846    0.0981    0.1766       469
           DoS     0.4932    0.1002    0.1665      3235
      Exploits     0.6062    0.9229    0.7317      8944
       Fuzzers     0.9043    0.8329    0.8671      4846
       Generic     0.9968    0.9878    0.9923     43219
Reconnaissance     0.9202    0.7444    0.8230      2696
     Shellcode     0.6302    0.6106    0.6203       321
         Worms     0.7692    0.6250    0.6897        32

      accuracy                         0.8972     64257
     macro avg     0.7589    0.5637    0.5901     64257
  weighted avg     0.9012    0.8972    0.8842     64257

